In [1]:
import h3
import pandas as pd
import geopandas as gpd
from shapely import geometry
import numpy as np

import pendulum

from sklearn.model_selection import GroupShuffleSplit

from count_modeling.constants import GCS_PATH, GROUPER_COL, MODEL_DATA_PATH
from count_modeling.utils import get_timestamp

In [2]:
INPUT_FILE_NAME = 'modeling_data_mar30.csv'

# 0. Data Read & Cleanup
Here we'll read in the data, drop a bunch of unnecessary columns, and also rename some columns to have cleaner names.

## Extract data

In [3]:
count_data = pd.read_csv(f'{GCS_PATH}/count_file/{INPUT_FILE_NAME}')

unwanted_columns = [
    'Unnamed: 0',
    'X', 'ID',
    'ATT.in.Million',
    'bgarea_t', 'bgcliparea_t', 'pctofbgarea_t',
    'bgarea_q', 'bgcliparea_q', 'pctofbgarea_q',
    'bgarea_h', 'bgcliparea_h', 'pctofbgarea_h',
    'rt_i_shd_tot_width', 'lt_i_shd_tot_width',
    'near_strava_id', 
    'ATT.in.Thousands',
    'rowIndex', 'primary', 'secondary', 'tertiary', 'residential', 'trunk', 'secondary_link', 'unclassified',
    'speed_0_25', 'speed_21_35', 'speed_greater_than_35',
    'path', 'bike_lane',
    'bike_route', 'cycle_track', 'trail', 'Interstate', 'Freeway', 'Principal_Arterial',
    'Minor_Arterial', 'Major_Collector', 'Minor_Collector', 'Local'
]
count_data = count_data.drop(columns=unwanted_columns)

count_data['centroid'] = count_data.apply(lambda x: geometry.Point(x['Long'], x['Lat']), axis=1)
count_data = gpd.GeoDataFrame(count_data, geometry='centroid', crs=4326)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


## Data cleanup

In [4]:
count_data['log_AADB'] = count_data['AADB'].apply(np.log)
count_data.query('AADB != 0')[['centroid', 'log_AADB', 'AADB']].explore('log_AADB')

In [5]:
count_data = count_data.rename(columns={
    'Stv_commute_adb': 'strava_commute_adb',
    'Stv_leisure_adb': 'strava_leisure_adb',
    'Stv_Ave_speed': 'strava_average_speed'})

count_data['total_lanes'] = count_data['rt_lanes_amt'] + count_data['lt_lanes_amt']
count_data['strava_leisure_pct'] = \
    count_data['strava_leisure_adb']/(count_data[['strava_leisure_adb', 'strava_commute_adb']].sum(axis=1))

# These Class V and Class VI values seem to be garbage
count_data['bike_facs'] = count_data['bike_facs'].replace(
    {'Not Collected': 'Unknown', 
     # 'Class V': 'Unknown',
     'Class VI': 'Unknown',
     '0': 'Unknown'
    })

In [6]:
count_data['bike_facs'] = count_data['bike_facs'].str.strip().fillna('Unknown')
count_data['fclass'] = count_data['fclass'].fillna('unknown')

In [7]:
county_summaries = count_data.groupby('county')['AADB'].agg(['mean', 'std']).rename(columns=lambda col: f'county_AADB_{col}')
# very large if we don't know better
county_summaries['county_AADB_std'] = county_summaries['county_AADB_std'].fillna(500)
count_data = count_data.join(county_summaries, on='county')

In [8]:
count_data['outlier'] = (count_data['AADB'] > (count_data['county_AADB_mean'] + 5 * count_data['county_AADB_std']))

In [9]:
count_data.outlier.value_counts()

False    4223
True       14
Name: outlier, dtype: int64

In [10]:
m = count_data.query('not outlier')[['tdg_id', 'year', 'AADB', 'outlier', 'centroid']].explore(color='blue')
m = count_data.query('outlier')[['tdg_id', 'year', 'AADB', 'outlier', 'centroid']].explore(color='red', m=m)
m

cool, let's go with 5 SD above the mean as our outlier threshold.

In [11]:
count_data['strava_ratio'] = count_data[['strava_commute_adb', 'strava_leisure_adb']].sum(axis=1) / count_data['AADB']

# deal with div by zero
count_data.loc[count_data.AADB == 0, 'strava_ratio'] = 0

In [12]:
count_data[['strava_ratio', 'strava_commute_adb', 'strava_leisure_adb', 'AADB', 'centroid']].query("strava_ratio > 1").explore('strava_ratio')

In [13]:
# Make some h3 Index values for stratification
count_data['h3_7'] = count_data.apply(lambda x: h3.geo_to_h3(x['Lat'], x['Long'], 7), axis=1)
count_data['h3_8'] = count_data.apply(lambda x: h3.geo_to_h3(x['Lat'], x['Long'], 8), axis=1)

## Make matrices

In [14]:
demographic_col_templates = [
    'popdensitysqmi_{}', 'hshlddensitysqmi_{}',
    'empnum_density_{}', 'pctwhite_{}', 'pctbiketowork_{}', 'pctatleastbachelors_{}', 
    'pctnoveh_{}']

demographic_cols = []
for distance in ('t', 'q', 'h'):
    demographic_cols += [col.format(distance) for col in demographic_col_templates]
    
strava_cols = ['strava_commute_adb', 'strava_leisure_adb', 'strava_average_speed', 'strava_leisure_pct']

data_mask = (~count_data.outlier) & (count_data.strava_ratio < 5)

count_data_clean = count_data[data_mask].reset_index(drop=True)

y_col = 'AADB'
x_cols = ['bike_facs', 
          # 'fclass',  - just use the official one
          'fc_draft', 'speed', 'slope', 
          # 'adt_amt', 'total_lanes', ## - these were included, but are almost entirely Null
          'near_univ_miles', 'near_large_univ_miles'] + strava_cols + demographic_cols 


X = count_data_clean[x_cols]
y = count_data_clean[y_col]


ADT and number of lanes are almost entirely missing data.

Everything else likely makes sense to fill with 0s.

In [16]:
X = X.fillna(0)

## Make train/test
Here we're going to use a grouped train/test split, grouped based on the `h3` index at resolution 7. This is to try to prevent information leakage from locations that are spatially adjacent. It's not perfect, but it should help (other than in edge cases of counts right on either side of a grid cell line).

In [17]:
TEST_SIZE = 0.2

grouper = count_data_clean[GROUPER_COL]

In [18]:
train_indx, test_indx = next(GroupShuffleSplit(random_state=42,test_size=TEST_SIZE).split(X, y, grouper))

X_train, X_test, y_train, y_test = X.loc[train_indx, :], X.loc[test_indx, :], y[train_indx], y[test_indx]

grouper_train = count_data_clean[GROUPER_COL][train_indx]

In [19]:
print(f"Test set is {len(test_indx)/(len(test_indx) + len(train_indx)) * 100 :.2f}% of sample. Target was {TEST_SIZE * 100.}%")

Test set is 19.86% of sample. Target was 20.0%


In [20]:
context_cols = [GROUPER_COL, 'centroid', 'location', 'year', 'no_of_months_data_collected', 'type']
context_train = count_data_clean.loc[train_indx, context_cols]
context_test = count_data_clean.loc[test_indx, context_cols]

## Write out count data to `gcs`

In [21]:
export_time = get_timestamp()
print(f"Timestamp for this verison is {export_time}")

Timestamp for this verison is 2023-05-25T06:22:48.356433-07:00


In [22]:
export_bundle = {
    'train': {'X': X_train, 'y': pd.DataFrame(y_train), 'context': context_train},
    'test': {'X': X_test, 'y': pd.DataFrame(y_test), 'context': context_test}
}

In [23]:
for part, tables in export_bundle.items():
    for table, df in tables.items():
        df.to_parquet(MODEL_DATA_PATH.format(version=export_time, part=part, table=table))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification doe